In [2]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
import pandas as pd
import numpy as np
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [4]:
from selenium import webdriver

In [17]:
from bs4 import BeautifulSoup
from urllib.request import build_opener, HTTPCookieProcessor, Request

sf_home_page = 'https://blockshopper.com/ca/san-francisco-county/cities/san-francisco'
opener = build_opener(HTTPCookieProcessor())
homepage_response = opener.open(sf_home_page, timeout=30)

In [18]:
master_df = pd.DataFrame()

In [19]:
home_page = BeautifulSoup(homepage_response, "html.parser")

In [20]:
all_links[23:]

['/ca/san-francisco-county/cities/san-francisco',
 '/ca/san-francisco-county/cities/san-leandro',
 '/ca/san-francisco-county/cities/san-pablo',
 '/ca/san-francisco-county/cities/san-pablo-unincorporated',
 '/ca/san-francisco-county/cities/san-ramon',
 '/ca/san-francisco-county/cities/walnut-creek',
 '/']

In [21]:
all_items = home_page.findAll("td", {'class': None})
all_links = []

for item in all_items:
    link = item.findNext('a')['href']
    if link not in all_links:
        all_links.append(link)

for link in all_links[23:]:
    to_open = "https://blockshopper.com" + link
    # to_open = "https://blockshopper.com/ca/los-angeles-county/cities/baldwin-park/streets/ahern-dr"
#     response = opener.open(to_open, timeout=30)  
    master_df = get_page(to_open, master_df)


Looking for [chromedriver 80.0.3987.16 mac64] driver in cache 
File found in cache by path [/Users/lisachen/.wdm/drivers/chromedriver/80.0.3987.16/mac64/chromedriver]

Looking for [chromedriver 80.0.3987.16 mac64] driver in cache 
File found in cache by path [/Users/lisachen/.wdm/drivers/chromedriver/80.0.3987.16/mac64/chromedriver]
missing middle school
{'address': ['1123 20th Street'], 'owner': ['Natalya Anchipolovsky and Arthur M Anchipolovsky'], 'purchase_price': ['$780,000\n(2005)'], 'property_tax': ['$9,850\n(2009)'], 'median_income': ['$137,610'], 'white': ['65%'], 'latin': ['13.5%'], 'black': ['4.9%'], 'native': ['N/A'], 'pacific_islander': ['2.3%'], 'asian': ['14.2%'], 'us_house_of_rep': ['State Sen. Mark Leno (D-03)'], 'state_senator': ['U.S. Rep. Nancy Pelosi (D-08)'], 'state_rep': ['State Assm. Tom Ammiano (D-13)'], 'elementary_school': 'Webster Elementary School', 'middle_school': '', 'high_school': 'San Francisco School District', 'neighbourhood': 'Potrero Hill', 'bedroo

KeyboardInterrupt: 

In [91]:
master_df.to_csv("scraped_property_san_mateo.csv")

In [64]:
# tester page

# to_open = "https://blockshopper.com/ca/los-angeles-county/baldwin-park/property/8535010017/13802-anada-street"
# response = opener.open(to_open, timeout=30) 
# property_page = BeautifulSoup(response, "html.parser")
# property_page.find_all("h2")[0].text.split(',')


['Los Angeles County', ' Baldwin Park', ' CA 91706']

In [8]:
def get_page(link, master_df):
    # open page with all the homes on this street
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(link)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    response = opener.open(link, timeout=30)  
    property_page = BeautifulSoup(response, "html.parser")
    all_homes = property_page.findAll("h4", {'class': 'card-title'})
    all_homes_links = find_links(all_homes)
    # open page with individual home information
    for home in all_homes_links:
        home_to_open = "https://blockshopper.com" + home
        page_driver = webdriver.Chrome(ChromeDriverManager().install())
        page_driver.get(home_to_open)
        page_driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        home_response = opener.open(home_to_open, timeout=30)      
        ind_property_page = BeautifulSoup(home_response, "html.parser")
        community_info = ind_property_page.findAll("p", {"class": "info-data"})
        master_df = create_property_row(ind_property_page, community_info, master_df)
        page_driver.close()
    try:
        print("trying to find next page")
        next_page = property_page.find_all("li", {"class":"page-item next_page"})[0].find("a")["href"]
        print("found link for next page")
        next_page_link = "https://blockshopper.com" + next_page
        print("about to recursively go to next page")
        driver.close()
        return get_page(next_page_link, master_df)
    except:
        print("reached the last page")
        return master_df

In [9]:
def find_links(all_items):
    all_links = []
    for item in all_items:
        link = item.findNext('a')['href']
        if link not in all_links:
            all_links.append(link)
    return all_links

In [10]:
def create_property_row(property_page, community_info, master_df):
    try:
        address = [property_page.findAll("h1")[0].text.strip()]
        owner = [community_info[0].text.strip()]
        purchase_price = [community_info[1].text.strip()]
        property_tax = [community_info[2].text.strip()]
        median_income = [community_info[3].text.strip()]
        white = [community_info[4].text.strip()]
        latin = [community_info[5].text.strip()]
        black = [community_info[6].text.strip()]
        native = [community_info[7].text.strip()]
        pacific_islander =[community_info[8].text.strip()]
        asian= [community_info[9].text.strip()]
        us_representatives = [community_info[10].text.strip()]
        state_senator =[community_info[11].text.strip()]
        state_rep = [community_info[12].text.strip()]
        neighbourhood = property_page.findAll("a", {"class": "col-xs-6 col-sm-4 col-md-6 col-lg-6"})[0].text.strip()
        lot_size = property_page.find(text="Lot Size").findNext('p').contents[0].strip()
        acre_tax = property_page.find(text="Tax $/Acre").findNext('p').contents[0].strip()
        home_size = property_page.find(text="Home Size").findNext('p').contents[0].strip()
        bedrooms = property_page.find(text="Beds").findNext('p').contents[0].strip()
        bathrooms = property_page.find(text="Baths").findNext('p').contents[0].strip()
        built_year = property_page.find(text="Built").findNext('p').contents[0].strip()
        address_details = property_page.find_all("h2")[0].text.split(',')
    
    except:
        print("not a correct property format")
        address = ""
        owner = ""
        purchase_price = ""
        property_tax = ""
        median_income = ""
        white = ""
        latin = ""
        black = ""
        native = ""
        pacific_islander =""
        asian= ""
        us_representatives = ""
        state_senator =""
        state_rep = ""
        neighbourhood = ""
        lot_size = ""
        acre_tax = ""
        home_size = ""
        bedrooms = ""
        bathrooms = ""
        built_year = ""
        address_details = ""
    
    try:
        county = address_details[0]
    except:
        print("no county")
        county = ""
    try:
        city = address_details[1]
    except:
        print("no city")
        city = ""
    try:
        postal = address_details[2]
    except:
        print("no postal")
        postal = ""

    try:
        middle_school = property_page.find(text="Middle School:").findNext('strong').text.strip()
    except: 
        print("missing middle school")
        middle_school = ""

    try:
        high_school = property_page.find(text="High School:").findNext('strong').text.strip()
    except: 
        print("missing high school")
        high_school = ""

    try:
        elementary_school = property_page.find(text="Elementary School:").findNext('strong').text.strip()
    except: 
        print("missing elementary school")
        elementary_school = ""
        
    community_dict = {'address': address, 'owner': owner, 'purchase_price': purchase_price,
                 'property_tax': property_tax, 'median_income':median_income,
                 'white':white, 'latin':latin, 'black':black, 'native':native, 'pacific_islander':pacific_islander,
                 'asian':asian, 'us_house_of_rep':us_representatives, 'state_senator':state_senator,
                 'state_rep':state_rep, 'elementary_school':elementary_school, 'middle_school':middle_school,
                 'high_school':high_school, 'neighbourhood':neighbourhood,
                 'bedrooms':bedrooms, 'bathrooms':bathrooms, "built_year":built_year,
                     "county":county, 'city':city,
                     'postal':postal, 'lot_size': lot_size, 'tax_per_acre': acre_tax,
                      'home_size':home_size
                     }
    print(community_dict)
    property_df = pd.DataFrame(community_dict)
    master_df = master_df.append(property_df)
    return master_df